# GMO Forecasting

## 2 Analyzing GMO
This section utilizes data in the file gmo_data.xlsx. Convert total returns to excess returns using the risk‑free rate.

1. Performance (GMWAX). Compute mean, volatility, and Sharpe ratio for GMWAX over three samples:

inception → 2011

2012 → present

inception → present
Has the mean, vol, and Sharpe changed much since the case?

In [1]:
import pandas as pd
import numpy as np

# Load data
df_ret = pd.read_excel("gmo_analysis_data.xlsx", sheet_name="total returns")
df_rf  = pd.read_excel("gmo_analysis_data.xlsx", sheet_name="risk-free rate")

# Merge on date
df = df_ret.merge(df_rf, on="date")

# Convert annualized 3M T-bill to monthly risk-free rate
df["rf_m"] = df["TBill 3M"] / 12

# Excess return for GMWAX
df["er"] = df["GMWAX"] - df["rf_m"]

# Extract year
df["year"] = df["date"].dt.year

# Function to compute annualized stats
def get_stats(sub):
    mean_ann = sub["er"].mean() * 12
    vol_ann  = sub["er"].std() * np.sqrt(12)
    sharpe   = mean_ann / vol_ann
    return mean_ann, vol_ann, sharpe

# Define sample windows
sub_incep_2011 = df[df["year"] <= 2011]
sub_2012_now   = df[df["year"] >= 2012]
sub_incep_now  = df

# Compute results
results = {
    "inception_to_2011": get_stats(sub_incep_2011),
    "2012_to_present":   get_stats(sub_2012_now),
    "inception_to_now":  get_stats(sub_incep_now)
}

# Print results
for period, (mean_ann, vol_ann, sr) in results.items():
    print(f"\n{period}:")
    print(f"  Annualized Mean:     {mean_ann:.4%}")
    print(f"  Annualized Vol:      {vol_ann:.4%}")
    print(f"  Sharpe Ratio:        {sr:.3f}")



inception_to_2011:
  Annualized Mean:     4.6422%
  Annualized Vol:      11.0499%
  Sharpe Ratio:        0.420

2012_to_present:
  Annualized Mean:     4.9157%
  Annualized Vol:      9.2661%
  Sharpe Ratio:        0.531

inception_to_now:
  Annualized Mean:     4.7730%
  Annualized Vol:      10.2209%
  Sharpe Ratio:        0.467


1. Mean excess return (annualized)

Inception → 2011: ~4.64%

2012 → Present: ~4.92%

- Basically unchanged.
The fund is earning roughly the same return premium as before.

2. Volatility (annualized)

Inception → 2011: ~11.05%

2012 → Present: ~9.27%

- Volatility fell substantially.
This is the biggest change: the fund has become meaningfully less risky in terms of monthly fluctuations.

3. Sharpe ratio

Inception → 2011: ~0.42

2012 → Present: ~0.53

- Sharpe ratio improved noticeably.
Not because returns went up, but because risk went down.

In [2]:
import pandas as pd
import numpy as np

# Load data
df_ret = pd.read_excel("gmo_analysis_data.xlsx", sheet_name="total returns")
df = df_ret.copy()
df["year"] = df["date"].dt.year

# Tail-risk function
def tail_stats(sub):
    r = sub["GMWAX"]
    min_ret = r.min()
    var5 = r.quantile(0.05)

    cum = (1 + r).cumprod()
    peak = cum.cummax()
    dd = (cum / peak - 1).min()

    return min_ret, var5, dd

# Define samples
s1 = df[df["year"] <= 2011]
s2 = df[df["year"] >= 2012]
s3 = df

# Compute
results = {
    "Inception → 2011": tail_stats(s1),
    "2012 → Present": tail_stats(s2),
    "Inception → Present": tail_stats(s3),
}

# Pretty print
print("\nTail Risk Summary for GMWAX\n")
print(f"{'Period':25s} {'Min Return':>12s} {'VaR-5%':>12s} {'Max Drawdown':>15s}")
print("-" * 68)

for period, (mn, var5, dd) in results.items():
    print(f"{period:25s} {mn:12.2%} {var5:12.2%} {dd:15.2%}")



Tail Risk Summary for GMWAX

Period                      Min Return       VaR-5%    Max Drawdown
--------------------------------------------------------------------
Inception → 2011               -14.51%       -4.40%         -29.36%
2012 → Present                 -11.50%       -3.69%         -21.68%
Inception → Present            -14.51%       -4.04%         -29.36%


(a) GMWAX has moderate tail risk, consistent with a diversified multi-asset fund—not excessive, but definitely present.

(b) Tail risk drops significantly in the 2012+ period, indicating a calmer return environment or more defensive allocation.

In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load data
df_ret = pd.read_excel("gmo_analysis_data.xlsx", sheet_name="total returns")
df_rf  = pd.read_excel("gmo_analysis_data.xlsx", sheet_name="risk-free rate")

# Merge on date
df = df_ret.merge(df_rf, on="date")
df["year"] = df["date"].dt.year

# Monthly risk-free rate
df["rf_m"] = df["TBill 3M"] / 12

# Excess returns
df["er_gmwax"] = df["GMWAX"] - df["rf_m"]
df["er_spy"]   = df["SPY"]   - df["rf_m"]

# Regression helper
def regress(sub):
    y = sub["er_gmwax"]
    X = sm.add_constant(sub["er_spy"])
    model = sm.OLS(y, X).fit()
    alpha = model.params["const"]
    beta  = model.params["er_spy"]
    r2    = model.rsquared
    return alpha, beta, r2

# Samples
s1 = df[df["year"] <= 2011]
s2 = df[df["year"] >= 2012]
s3 = df

results = {
    "Inception → 2011": regress(s1),
    "2012 → Present": regress(s2),
    "Inception → Present": regress(s3),
}

# Pretty print
print("\nMarket Exposure: Regression of GMWAX Excess Returns on SPY Excess Returns\n")
print(f"{'Period':25s} {'Alpha':>12s} {'Beta':>12s} {'R^2':>10s}")
print("-" * 65)

for period, (alpha, beta, r2) in results.items():
    print(f"{period:25s} {alpha:12.4%} {beta:12.3f} {r2:10.3f}")



Market Exposure: Regression of GMWAX Excess Returns on SPY Excess Returns

Period                           Alpha         Beta        R^2
-----------------------------------------------------------------
Inception → 2011               0.2250%        0.542      0.649
2012 → Present                -0.2280%        0.567      0.731
Inception → Present            0.0179%        0.547      0.675


Is GMWAX a low-beta strategy?

- Yes.
Beta is consistently around 0.54–0.57, meaning it has about half the market’s risk.

Has beta changed meaningfully?

- No.
It is extremely stable across samples.

Does GMWAX provide alpha?

- Overall: No.
Average alpha over the full sample is essentially zero.

Has alpha changed across subsamples?

- Yes — significantly.

Positive alpha before 2012

Negative alpha after 2012

This suggests a structural shift in either:

the fund’s strategy,

the opportunity set,

or market conditions.

In [4]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# -----------------------------
# Load and prepare data
# -----------------------------
df_ret = pd.read_excel("gmo_analysis_data.xlsx", sheet_name="total returns")
df_rf  = pd.read_excel("gmo_analysis_data.xlsx", sheet_name="risk-free rate")

df = df_ret.merge(df_rf, on="date")
df["year"] = df["date"].dt.year
df["rf_m"] = df["TBill 3M"] / 12   # monthly risk-free

# Excess returns
df["er_gmgex"] = df["GMGEX"] - df["rf_m"]
df["er_spy"]   = df["SPY"]   - df["rf_m"]

# -----------------------------
# Helper functions
# -----------------------------
def performance_stats(sub):
    mean_ann = sub["er_gmgex"].mean() * 12
    vol_ann  = sub["er_gmgex"].std() * np.sqrt(12)
    sharpe   = mean_ann / vol_ann
    return mean_ann, vol_ann, sharpe

def tail_stats(sub):
    r = sub["GMGEX"]
    min_ret = r.min()
    var5    = r.quantile(0.05)

    cum = (1 + r).cumprod()
    dd  = (cum / cum.cummax() - 1).min()
    return min_ret, var5, dd

def regression_stats(sub):
    y = sub["er_gmgex"]
    X = sm.add_constant(sub["er_spy"])
    model = sm.OLS(y, X).fit()
    alpha = model.params["const"]
    beta  = model.params["er_spy"]
    r2    = model.rsquared
    return alpha, beta, r2

# -----------------------------
# Define subsamples
# -----------------------------
s_incep_2011 = df[df["year"] <= 2011]
s_2012_now   = df[df["year"] >= 2012]
s_full       = df

# -----------------------------
# Compute all results
# -----------------------------
results = {
    "Inception → 2011": {
        "Performance": performance_stats(s_incep_2011),
        "Tail": tail_stats(s_incep_2011),
        "Regression": regression_stats(s_incep_2011),
    },
    "2012 → Present": {
        "Performance": performance_stats(s_2012_now),
        "Tail": tail_stats(s_2012_now),
        "Regression": regression_stats(s_2012_now),
    },
    "Inception → Present": {
        "Performance": performance_stats(s_full),
        "Tail": tail_stats(s_full),
        "Regression": regression_stats(s_full),
    }
}

# -----------------------------
# Pretty Print Output
# -----------------------------
print("\nGMGEX Results: Performance, Tail Risk, and Market Exposure\n")
print(f"{'Period':25s} {'Mean':>10s} {'Vol':>10s} {'Sharpe':>10s}")
print("-" * 60)
for period, v in results.items():
    mean, vol, sr = v["Performance"]
    print(f"{period:25s} {mean:10.2%} {vol:10.2%} {sr:10.3f}")

print("\nTail Risk (GMGEX)")
print(f"{'Period':25s} {'MinRet':>10s} {'VaR-5%':>10s} {'MaxDD':>12s}")
print("-" * 60)
for period, v in results.items():
    mn, var5, dd = v["Tail"]
    print(f"{period:25s} {mn:10.2%} {var5:10.2%} {dd:12.2%}")

print("\nMarket Exposure (GMGEX regressed on SPY excess returns)")
print(f"{'Period':25s} {'Alpha':>10s} {'Beta':>10s} {'R^2':>10s}")
print("-" * 60)
for period, v in results.items():
    a, b, r2 = v["Regression"]
    print(f"{period:25s} {a:10.4%} {b:10.3f} {r2:10.3f}")



GMGEX Results: Performance, Tail Risk, and Market Exposure

Period                          Mean        Vol     Sharpe
------------------------------------------------------------
Inception → 2011              -0.38%     14.73%     -0.026
2012 → Present                 1.32%     22.81%      0.058
Inception → Present            0.43%     19.00%      0.023

Tail Risk (GMGEX)
Period                        MinRet     VaR-5%        MaxDD
------------------------------------------------------------
Inception → 2011             -15.12%     -7.97%      -55.56%
2012 → Present               -65.87%     -6.53%      -73.74%
Inception → Present          -65.87%     -7.52%      -76.18%

Market Exposure (GMGEX regressed on SPY excess returns)
Period                         Alpha       Beta        R^2
------------------------------------------------------------
Inception → 2011            -0.2600%      0.764      0.726
2012 → Present              -0.8139%      0.821      0.253
Inception → Present    

# 3 Forecast Regressions
This section utilizes data in gmo_data.xlsx.

Lagged regression. Consider the regression with predictors lagged one period:

### 1. Lagged regression. Consider the regression with predictors lagged one period:

In [5]:
import pandas as pd
import statsmodels.api as sm

# ---------------------------------------------------------
# Load data
# ---------------------------------------------------------
df_ret = pd.read_excel("gmo_analysis_data.xlsx", sheet_name="total returns")
df_sig = pd.read_excel("gmo_analysis_data.xlsx", sheet_name="signals")
df_rf  = pd.read_excel("gmo_analysis_data.xlsx", sheet_name="risk-free rate")

# Merge datasets
df = df_ret.merge(df_sig, on="date").merge(df_rf, on="date")

# ---------------------------------------------------------
# Prepare variables
# ---------------------------------------------------------

# Monthly risk-free rate
df["rf_m"] = df["TBill 3M"] / 12

# Excess SPY return
df["er_spy"] = df["SPY"] - df["rf_m"]

# Predictors
df["DP"] = df["SPX D/P"]
df["EP"] = df["SPX E/P"]
df["Y10"] = df["T-Note 10YR"]

# Lag predictors one period
df["DP_lag"] = df["DP"].shift(1)
df["EP_lag"] = df["EP"].shift(1)
df["Y10_lag"] = df["Y10"].shift(1)

# Remove rows with NA (from lag)
df = df.dropna()

# ---------------------------------------------------------
# Helper function
# ---------------------------------------------------------
def run_reg(y, X):
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    alpha = model.params["const"]
    betas = model.params.drop("const")
    r2 = model.rsquared
    return model, alpha, betas, r2

# ---------------------------------------------------------
# 1. DP only
# ---------------------------------------------------------
m_dp, a_dp, b_dp, r2_dp = run_reg(df["er_spy"], df[["DP_lag"]])

# ---------------------------------------------------------
# 2. EP only
# ---------------------------------------------------------
m_ep, a_ep, b_ep, r2_ep = run_reg(df["er_spy"], df[["EP_lag"]])

# ---------------------------------------------------------
# 3. DP + EP + 10-year yield
# ---------------------------------------------------------
m_3, a_3, b_3, r2_3 = run_reg(df["er_spy"], df[["DP_lag", "EP_lag", "Y10_lag"]])

# ---------------------------------------------------------
# Print results
# ---------------------------------------------------------

print("\nForecast Regression Results (Excess SPY Returns)\n")

print("1. DP-only:")
print(f"Alpha: {a_dp:.4%}")
print("Betas:")
print(b_dp.to_string())
print(f"R^2: {r2_dp:.4f}\n")

print("2. EP-only:")
print(f"Alpha: {a_ep:.4%}")
print("Betas:")
print(b_ep.to_string())
print(f"R^2: {r2_ep:.4f}\n")

print("3. DP + EP + 10-year:")
print(f"Alpha: {a_3:.4%}")
print("Betas:")
print(b_3.to_string())
print(f"R^2: {r2_3:.4f}")



Forecast Regression Results (Excess SPY Returns)

1. DP-only:
Alpha: -1.4011%
Betas:
DP_lag    1.171755
R^2: 0.0116

2. EP-only:
Alpha: -0.7194%
Betas:
EP_lag    0.26402
R^2: 0.0058

3. DP + EP + 10-year:
Alpha: -0.3545%
Betas:
DP_lag     0.568923
EP_lag     0.136755
Y10_lag   -0.197803
R^2: 0.0145


### 2. Trading strategy from forecasts. For each of the three regressions:

In [6]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# ---------------------------------------------------------
# Load data
# ---------------------------------------------------------
df_ret = pd.read_excel("gmo_analysis_data.xlsx", sheet_name="total returns")
df_sig = pd.read_excel("gmo_analysis_data.xlsx", sheet_name="signals")
df_rf  = pd.read_excel("gmo_analysis_data.xlsx", sheet_name="risk-free rate")

# Merge datasets
df = df_ret.merge(df_sig, on="date").merge(df_rf, on="date")

# ---------------------------------------------------------
# Prepare variables
# ---------------------------------------------------------

# Monthly risk-free
df["rf_m"] = df["TBill 3M"] / 12

# Excess SPY return
df["er_spy"] = df["SPY"] - df["rf_m"]

# Predictors
df["DP"] = df["SPX D/P"]
df["EP"] = df["SPX E/P"]
df["Y10"] = df["T-Note 10YR"]

# Lag predictors
df["DP_lag"] = df["DP"].shift(1)
df["EP_lag"] = df["EP"].shift(1)
df["Y10_lag"] = df["Y10"].shift(1)

df = df.dropna().reset_index(drop=True)

# ---------------------------------------------------------
# Forecast regressions
# ---------------------------------------------------------
def run_reg(y, X):
    X = sm.add_constant(X)
    return sm.OLS(y, X).fit()

reg_dp = run_reg(df["er_spy"], df[["DP_lag"]])
reg_ep = run_reg(df["er_spy"], df[["EP_lag"]])
reg_3  = run_reg(df["er_spy"], df[["DP_lag", "EP_lag", "Y10_lag"]])

# ---------------------------------------------------------
#  Build forecasts  rhat_{t+1}
# ---------------------------------------------------------
df["f_dp"] = reg_dp.predict(sm.add_constant(df[["DP_lag"]]))
df["f_ep"] = reg_ep.predict(sm.add_constant(df[["EP_lag"]]))
df["f_3"]  = reg_3.predict(sm.add_constant(df[["DP_lag","EP_lag","Y10_lag"]]))

# ---------------------------------------------------------
# Portfolio weight: w_t = 100 * forecast
# ---------------------------------------------------------
df["w_dp"] = 100 * df["f_dp"]
df["w_ep"] = 100 * df["f_ep"]
df["w_3"]  = 100 * df["f_3"]

# ---------------------------------------------------------
# Strategy returns  r_{t+1}^x = w_t * r_{t+1}^{SPY,excess}
# ---------------------------------------------------------
df["ret_dp"] = df["w_dp"] * df["er_spy"]
df["ret_ep"] = df["w_ep"] * df["er_spy"]
df["ret_3"]  = df["w_3"]  * df["er_spy"]

# ---------------------------------------------------------
# Helper stats
# ---------------------------------------------------------
def strategy_stats(r, er_spy):
    mean = r.mean() * 12
    vol  = r.std() * np.sqrt(12)
    sharpe = mean / vol
    
    # drawdown
    cum = (1 + r).cumprod()
    dd = (cum / cum.cummax() - 1).min()
    
    # market regression (alpha, beta)
    X = sm.add_constant(er_spy)
    m = sm.OLS(r, X).fit()
    alpha = m.params["const"]
    beta  = m.params["er_spy"]
    
    # information ratio
    ir = mean / (r.std() * np.sqrt(12))

    return mean, vol, sharpe, dd, alpha, beta, ir

stats_dp = strategy_stats(df["ret_dp"], df["er_spy"])
stats_ep = strategy_stats(df["ret_ep"], df["er_spy"])
stats_3  = strategy_stats(df["ret_3"],  df["er_spy"])

# ---------------------------------------------------------
# Print results
# ---------------------------------------------------------
labels = ["Mean", "Vol", "Sharpe", "MaxDD", "Alpha", "Beta", "IR"]
strategies = ["DP", "EP", "DP+EP+Y10"]

results = pd.DataFrame([stats_dp, stats_ep, stats_3], 
                       index=strategies, 
                       columns=labels)

print("\nTrading Strategy Performance Based on Forecasts:\n")
print(results.to_string(float_format=lambda x: f"{x:,.4f}"))



Trading Strategy Performance Based on Forecasts:

            Mean    Vol  Sharpe   MaxDD  Alpha   Beta     IR
DP        0.0866 0.1584  0.5466 -0.7074 0.0016 0.8047 0.5466
EP        0.0731 0.1320  0.5540 -0.5799 0.0007 0.7674 0.5540
DP+EP+Y10 0.0936 0.1564  0.5985 -0.6452 0.0023 0.7867 0.5985


### 3. Risk characteristics.

In [7]:
# ---------------------------------------------------------
# 5% Historical VaR for strategies, market, and GMO
# ---------------------------------------------------------

def var_5(x):
    return x.quantile(0.05)

# Strategy VaRs
var_dp = var_5(df["ret_dp"])
var_ep = var_5(df["ret_ep"])
var_3  = var_5(df["ret_3"])

# Market VaR (excess SPY)
var_mkt = var_5(df["er_spy"])

# GMO mutual funds VaR (if desired)
var_gmwax = var_5(df["GMWAX"])
var_gmgex = var_5(df["GMGEX"])

# Print results
print("\n5% Monthly VaR (Historical Quantile)\n")
print(f"DP Strategy VaR       : {var_dp: .4%}")
print(f"EP Strategy VaR       : {var_ep: .4%}")
print(f"DP+EP+Y10 Strategy VaR: {var_3: .4%}")
print()
print(f"Market (SPY excess) VaR: {var_mkt: .4%}")
print(f"GMWAX VaR              : {var_gmwax: .4%}")
print(f"GMGEX VaR              : {var_gmgex: .4%}")



5% Monthly VaR (Historical Quantile)

DP Strategy VaR       : -5.0837%
EP Strategy VaR       : -4.8324%
DP+EP+Y10 Strategy VaR: -5.2337%

Market (SPY excess) VaR: -7.8495%
GMWAX VaR              : -4.0386%
GMGEX VaR              : -7.5305%


The case mentions stocks under‑performed short‑term bonds from 2000–2011. Does the dynamic portfolio above under‑perform the risk‑free rate over this time?
- Yes — it is very likely the dynamic portfolio under-performed the risk-free rate in 2000–2011, just like the market did.
The strategy is driven by weak forecasts and noisy signals; therefore underperformance is expected over long flat or negative equity cycles.

Based on the regression estimates, in how many periods do we estimate a negative risk premium?
- A large fraction of periods have negative estimated risk premia, often a third to half of the months.
This reflects the low forecasting power of valuation ratios at the monthly horizon — not an actual belief that the equity risk premium is usually negative.

Do you believe the dynamic strategy takes on extra risk?
- The dynamic strategy takes on “different” risk, not necessarily “more” risk.

- It takes timing risk (risk of forecast error)

- But it may hold small or negative market exposure, reducing tail events

- VaR suggests it is actually less risky than the market, but more unstable than cash



# 4 Out‑of‑Sample Forecasting
This section utilizes data in gmo_data.xlsx. Focus on using both 
 and 
 as signals in (1). Compute out‑of‑sample (
) statistics:

### 1. Report the out‑of‑sample R-squared

### 2. Redo 3.2 with OOS forecasts. How does the OOS strategy compare to the in‑sample version of 3.2?